In [ ]:
import cv2
import numpy as np
import string
import datetime

In [ ]:
debug = False

In [ ]:
def shutdown(cam):
    cv2.destroyAllWindows()
    cam.release()

In [ ]:
def extractRect(img, x, y, w, h):
    if img is None:
        return None
    
    return img[y:y+h,x:x+w]

In [ ]:
def capture(cam):
    raw = cam.read()[1]
    
    if raw is None:
        return None
    
    x = 320
    y = 0
    w = 290
    h = 480
    
    half = extractRect(raw, x, y, w, h)
        
    gray = cv2.cvtColor(half, cv2.COLOR_RGB2GRAY)
    
    return gray

In [ ]:
def drawRect(img, x, y, w, h):
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,0),1)

In [ ]:
def threshold(img):
    retval, threshold = cv2.threshold(img, 150, 255, cv2.THRESH_OTSU)
    
    return threshold

In [ ]:
def allEqual(aList):
    return all(aList[0] == item for item in aList)

In [ ]:
cam = cv2.VideoCapture("file:///Users/jackhig/Desktop/meter_qt.mp4")
# cam = cv2.VideoCapture(0)

triggerZoneSize = 20
triggerZones = []
triggerStates = []
fired = -1

triggerQty = 0.5
total = 0.0

def addTrigger(x, y):
    triggerZones.append((x, y, triggerZoneSize))
    triggerStates.append(-1)
    
addTrigger(260, 190)
addTrigger(270, 230)
addTrigger(260, 270)

alphabet = string.ascii_lowercase

while True:
    liveImg = capture(cam)

    if liveImg is None:
        shutdown(cam)
        break
    
    thresholded = threshold(liveImg)
    
    thresholded_annotated = None
    if debug:
        cv2.imshow('thresh', thresholded)
        thresholded_annotated = threshold(liveImg)
    
    zoneCount = 0
    
    for zone in triggerZones:
        rect = extractRect(thresholded, zone[0], zone[1], zone[2], zone[2])
        drawRect(liveImg, zone[0], zone[1], zone[2], zone[2])
        
        if debug:
            drawRect(thresholded_annotated, zone[0], zone[1], zone[2], zone[2])
            
        blackOrWhite = 0
        if cv2.mean(rect)[0] > 128.0:
            blackOrWhite = 1
        else:
            blackOrWhite = 0
            
        triggerStates[zoneCount] = blackOrWhite
        
        if debug:
            cv2.imshow(alphabet[zoneCount], rect)
        zoneCount = zoneCount+1
    
    if allEqual(triggerStates):
        if triggerStates[-1] is not fired and triggerStates[0] is not fired:
            if fired is -1:
                print('starting')
            else:
                now = datetime.datetime.now()
                total = total + triggerQty
                print(now, ":", total)
            fired = triggerStates[-1]
            
    
    if debug:
        cv2.imshow( 'live', liveImg )
        cv2.imshow( 'thresh', thresholded_annotated )


    key = cv2.waitKey(10)
    if key == 27:
        shutdown(cam)
        break

print ("Done.")